In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Engineering/Curriculum/8th Semester/Internship/descriptive_evaluation_project/Hewlett Dataset

/content/drive/.shortcut-targets-by-id/17Gn89Edqfyxljr8tO09VdcQWGcUCa_Ua/descriptive_evaluation_project/Hewlett Dataset


In [ ]:
       
    # lst = []
    # TRAIN_FILE_NAME = 'Hewlett Dataset.csv'
    # print("Reading training data...")
    # df = pd.read_csv(TRAIN_FILE_NAME)
    # df.dropna(inplace=True)
    # start = df["EssaySet"]
    # # print(type(start))
    # startText = df["EssayText"]
    # count=0
    # for row in df.iterrows():
    #     # print(type(str(row[1][1])), start)
    #     if(str(row[1][1])==str(start) and count!=0):
    #         lst.append([startText, row[1][4], 1])
    #     else:
    #         start = row[1][1]
    #         startText = row[1][4]
    #         count = 0
    #     count+=1
    # start = df["EssaySet"]
    # startText = df["EssayText"]
    # for row in df.iterrows():
    #     for row_in in df.iterrows():
    #         # print(type(str(row[1][1])), start)
    #         if(str(row_in[1][1])!=str(row[1][1])):
    #             lst.append([row[1][4], row_in[1][4], 0])
    # # lst[800:900]
    # df_main = pd.DataFrame(lst, columns =['question1', 'question2', 'is_duplicate'], dtype = int) 
    # print(df_main) 

    # df_main = df_main.sample(frac=1, random_state=42).reset_index(drop=True)
    # df_main
    # df_main.to_csv('train_main.csv')
import pandas as pd
TRAIN_FILE_NAME = 'train_oop.csv'
df_main = pd.read_csv(TRAIN_FILE_NAME, encoding='latin1')
df_main

,question1,question2,is_duplicate
0,Dynamic (run time) polymorphism is the polymor...,Constructor is piece of code which we are usin...,0
1,A virtual function or virtual method in an OOP...,"Within a single program, output of a function ...",0
2,new: Allocates memory for the object on the fr...,Object is an instance of a class. An object in...,0
3,Overloading occurs when two or more methods in...,OOPs allows us to hide implementation details ...,0
4,An interface is most certainly not a blueprint...,An interface is better than a abstract class w...,0
...,...,...,...
211533,OOPs allows us to hide implementation details ...,"Exception is an abnormal condition. In Java, a...",0
211534,"When Java encounters an exception, it throws i...","An interface is a completely ""abstract class"" ...",0
211535,Copy constructor is called when a new object i...,Overriding occurs when two methods have the sa...,0
211536,Dynamic Polymorphism(Late Binding/ Runtime Pol...,The interface consists of the signatures of me...,0


In [ ]:
# Import required libraries
import tensorflow as tf
import numpy as np


# File paths
EMBEDDING_FILE_PATH = 'glove.6B.100d.txt'

# Model parameters
maxlen = 15             # Maximum number of words in a sentence
n_units = 50            # Number of units in LSTM layer
clipnorm = 1.5          # Norm for gradient clipping
EMBEDDING_DIM = 100     # Dimension of embedding vectors


# Function to calculate Manhattan LSTM distance
def manh_lstm_distance(question1, question2):
  distance = tf.keras.backend.abs(question1-question2)
  distance = tf.keras.backend.sum(distance, axis=1, keepdims=True)
  distance = -distance
  distance = tf.keras.backend.exp(distance)
  return distance

# Function to create Embedding layer using Tokenizer object
def create_embedding_layer(tokenizer):

    # Create a dictionary mapping vocabulary words to embedding vectors
    embeddings_index = {}
    f = open(EMBEDDING_FILE_PATH, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    # Create embedding matrix to initialize Embedding layer
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    # Create Embedding layer
    embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix)
    embedding_layer = tf.keras.layers.Embedding(len(word_index)+1,
                                                EMBEDDING_DIM,
                                                embeddings_initializer=embeddings_initializer,
                                                input_length=maxlen,
                                                trainable=True,
                                                name="embedding_layer")

    return embedding_layer

# Function to create Siamese Manhattan LSTM model
def create(tokenizer):
    print("Creating model...")
    embedding_layer = create_embedding_layer(tokenizer)
    question1 = tf.keras.layers.Input(shape=(maxlen,), dtype='int32', name="question1")
    question2 = tf.keras.layers.Input(shape=(maxlen,), dtype='int32', name="question2")
    question1_encoded = embedding_layer(question1)
    question2_encoded = embedding_layer(question2)
    common_lstm_layer = tf.keras.layers.LSTM(n_units, name="common_lstm_layer")
    question1_output = common_lstm_layer(question1_encoded)
    question2_output = common_lstm_layer(question2_encoded)
    manhattan_lstm_distance = tf.keras.layers.Lambda(lambda x: manh_lstm_distance(x[0], x[1]), name="manhattan_lstm_distance")([question1_output, question2_output])
    model = tf.keras.models.Model([question1, question2], manhattan_lstm_distance)
    loss = 'binary_crossentropy'
    optimizer = tf.keras.optimizers.Adam(clipnorm=clipnorm)
    metrics = ['accuracy', 'mse']
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

In [ ]:
# Import required libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

# Prerequisites for cleaning
nltk.download("stopwords")                      # Download stopwords from NLTK library
nltk.download('wordnet')                        # Download wordnet, a lexixal database from NLTK library
stopwords = set(stopwords.words('english'))     # Store stopwords
lemmatizer = WordNetLemmatizer()                # Create object for lemmatization

# Function for standard cleaning of text (remove punctuations, abbreviations, etc.) using regular expressions
def standard_clean(text):
  text = str(text)
  text = text.lower()
  text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "cannot ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r",", " ", text)
  text = re.sub(r"\.", " ", text)
  text = re.sub(r"!", " ! ", text)
  text = re.sub(r"\/", " ", text)
  text = re.sub(r"\^", " ^ ", text)
  text = re.sub(r"\+", " + ", text)
  text = re.sub(r"\-", " - ", text)
  text = re.sub(r"\=", " = ", text)
  text = re.sub(r"'", " ", text)
  text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
  text = re.sub(r":", " : ", text)
  text = re.sub(r" e g ", " eg ", text)
  text = re.sub(r" b g ", " bg ", text)
  text = re.sub(r" u s ", " american ", text)
  text = re.sub(r"\0s", "0", text)
  text = re.sub(r" 9 11 ", "911", text)
  text = re.sub(r"e - mail", "email", text)
  text = re.sub(r"j k", "jk", text)
  text = re.sub(r"\s{2,}", " ", text)
  return text

# Function to remove stopwords from a sentence
def remove_stopwords(text):
  text = text.split()
  clean = ""
  for w in text:
    if w not in stopwords:
      clean = clean + " " + w
  return str(clean[1:])

# Function to lemmatize words of a sentence using Lemmatizer object
def lemmatize(text):
  text = text.split()
  clean = ""
  for w in text:
    clean = clean + " " + lemmatizer.lemmatize(w)
  return str(clean[1:])

# Function to clean the text
def clean(text):
  text = standard_clean(text)
  text = remove_stopwords(text)
  text = lemmatize(text)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Import required libraries
import tensorflow as tf
import numpy as np
import pickle           # Library to load Tokenizer object

# Import local modules
# import clean_text as ct

# Preprocessing parameters
maxlen=15        # Maximum number of words in a processed question


# Function to create Tokenizer object
def tokenize(df):
    df['concatenated'] = df['question1'] + " " + df['question2']
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(df.concatenated)
    return tokenizer

# Function to preprocess textual data
def preprocess(df, mode):

    # Clean the text
    df['question1'] = df.question1.map(lambda x: clean(x))
    df['question2'] = df.question2.map(lambda x: clean(x))

    # Prepare the data for the model
    print("Preparing data for model...")

    # While training, create Tokenizer object and also return labels
    if mode=='train':
        tokenizer = tokenize(df)
        df['question1'] = tokenizer.texts_to_sequences(df.question1)
        df['question2'] = tokenizer.texts_to_sequences(df.question2)
        question1 = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(df.question1, maxlen=maxlen)))
        question2 = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(df.question2, maxlen=maxlen)))
        labels = np.array(list(df.is_duplicate))
        return question1, question2, labels, tokenizer

    # While predicting, load existing Tokenizer object
    if mode=='predict':
        with open('../checkpoints/tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(handle)
        df['question1'] = tokenizer.texts_to_sequences(df.question1)
        df['question2'] = tokenizer.texts_to_sequences(df.question2)
        question1 = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(df.question1, maxlen=maxlen)))
        question2 = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(df.question2, maxlen=maxlen)))
        return question1, question2

In [ ]:
# Import required libraries
import pandas as pd

# Import local modules
# import preprocess_data as pp

# File paths
# DATASET_PATH = '../dataset'
TRAIN_FILE_NAME = 'train_oop.csv'

# Function to load, process training data and create Tokenizer onject
def load():
    print("Reading training data...")
    df = pd.read_csv(TRAIN_FILE_NAME, encoding='latin1')

    print("Preprocessing training data...")
    question1, question2, labels, tokenizer = preprocess(df, mode='train')

    return question1, question2, labels, tokenizer

In [ ]:
# Library to save Tokenizer object
import pickle

# Import local modules
# import load_data as ld
# import create_model as cm

# File paths
CHECKPOINT_PATH = '../checkpoints'
WEIGHTS_FILE_NAME = '/weights'
MODEL_FILE_NAME = '/model.h5'
TOKENIZER_FILE_NAME = '/tokenizer.pickle'

# Training parameters
epochs = 2                  # Number of epochs
batch_size = 64             # Training batch size
validation_split=0.2        # Fraction of training data for validation
verbose=1                   # Show progress bar

# Load, process training data and create Tokenizer onject
question1, question2, labels, tokenizer = load()

# Create model using Tokenizer object
model = create(tokenizer)

# Train the model
print("Training model...")
model.fit([question1, question2], labels, epochs=epochs, batch_size=batch_size, validation_split=validation_split, verbose=verbose)

# Save model and model weights
print("Saving model and model weights...")
model.save(CHECKPOINT_PATH + MODEL_FILE_NAME)
model.save_weights(CHECKPOINT_PATH + WEIGHTS_FILE_NAME)

# Save Tokenizer object
print("Saving Tokenizer object...")
with open(CHECKPOINT_PATH + TOKENIZER_FILE_NAME, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Reading training data...
Preprocessing training data...
Preparing data for model...
Creating model...
Training model...
Epoch 1/2
2645/2645 [==============================] - 95s 34ms/step - loss: 0.0127 - accuracy: 0.9980 - mse: 0.0020 - val_loss: 0.0133 - val_accuracy: 0.9978 - val_mse: 0.0021
Epoch 2/2
2645/2645 [==============================] - 86s 32ms/step - loss: 0.0096 - accuracy: 0.9980 - mse: 0.0019 - val_loss: 0.0125 - val_accuracy: 0.9978 - val_mse: 0.0020
Saving model and model weights...
Saving Tokenizer object...


In [ ]:
# Import required libraries
import tensorflow as tf
import pandas as pd

# from create_model import manh_lstm_distance

# File paths
CHECKPOINT_PATH = '../checkpoints'
MODEL_FILE_NAME = '/model.h5'
DATASET_PATH = '../dataset'
TEST_FILE_NAME = "train_main.csv"

# Prediction parameters
skiprows = 0            # Predict labels for question pairs from index 'skiprows'
nrows = 2000           # to index 'skiprows + nrows' in the test file

threshold = 0.001         # Minimum Manhattan LSTM distance between two outputs
                        # for them to be classified as semantically similar


# Load trained model
print("Loading model...")
model = tf.keras.models.load_model(CHECKPOINT_PATH + MODEL_FILE_NAME, custom_objects={"manh_lstm_distance": manh_lstm_distance})

# Read test file
print("Reading test data...")
df = pd.read_csv(TEST_FILE_NAME, skiprows=skiprows, nrows=nrows)
l1 = df['question1'].tolist()
l2 = df['question2'].tolist()
print(df['question1'])
print(df['question2'])
# Preprocess test data
print("Preprocessing test data...")
question1, question2 = preprocess(df, mode='predict')

# Predict Manhattan LSTM distances
print("Predicting Manhattan LSTM distances...")
manh_lstm_distance = model.predict([question1, question2], verbose=1)

# Make binary predictions
print("Making binary predictions...")
prediction = manh_lstm_distance>threshold
prediction = prediction.astype(int)

# Print predictions
score = manh_lstm_distance*1000
data = {'Manhattan LSTM distances': list(manh_lstm_distance), 'Score': list(score), 'Prediction': list(prediction)}
df1 = pd.DataFrame(data)
print(df1)

Loading model...
Reading test data...
0       First you would need to know what the samples ...
1       You would need the starting mass, ending mass,...
2       To replicate these students experiment you wou...
3       To replicate the experiment you would need to ...
4       The additonal information youwould need to kno...
                              ...                        
1995    To replicate this experiment I would need to k...
1996    The additional information I would need to rep...
1997    The procedure could be better  if they include...
1998    After reading this procedure i think it would ...
1999    Pour vinegar in each of four separate, but ide...
Name: question1, Length: 2000, dtype: object
0       To replicate the experiments you will need all...
1       1. Have a determined size for the starting sam...
2       After reading the groups procedure the additio...
3       You will need the type of vinegar and know the...
4       You need to add to number 4, it should 